## 进行数据过滤

数据过滤：

- 只保留 fold == 'all-folds'和test 的记录，忽略 Fold-1 到 Fold-5，以聚焦跨折叠的平均性能。

- 保留所有方法，包括基于物理的（Peak、fft）和监督学习的（ResNet、Transformer、Mamba2、inception_time 等）。

- 按 task（hr、resp Gonzalo_rr、spo2、BP_sys、BP_dia）和 ring_type（ring1、ring2）分组，提取 mae、rmse、mape、pearson 等指标。

In [ ]:
# 导入必要的库
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
import os

# 1. 加载
file_path = "/root/RingTool/output/all_results.csv"
raw_df = pd.read_csv(file_path)

# 10. 创建输出文件夹
output_dir = 'output'
os.makedirs(output_dir, exist_ok=True)

# 2. 过滤
# 更新过滤规则：
# - mode == 'test' 或 '5fold'
# - fold == 'all-folds' 或 fold 是 NaN（物理方法）
mask_mode = raw_df['mode'].isin(['test', '5fold'])
mask_fold = (raw_df['fold'] == 'all-folds') | (raw_df['fold'].isna())
df = raw_df[mask_mode & mask_fold].copy()

# 3. 检查过滤结果
print(f"原始数据行数: {len(raw_df)}")
print(f"过滤后数据行数: {len(df)}")
print("\nMode 分布：")
print(df['mode'].value_counts())
print("\nFold 分布：")
print(df['fold'].value_counts(dropna=False))
print("\n方法分布：")
print(df['method_name'].value_counts())

原始数据行数: 513
过滤后数据行数: 117

Mode 分布：
test     101
5fold     16
Name: mode, dtype: int64

Fold 分布：
NaN          74
all-folds    43
Name: fold, dtype: int64

方法分布：
peak              36
fft               36
inception_time    15
transformer       14
resnet            14
ratio              2
Name: method_name, dtype: int64


In [2]:
# 4. 去重
df_deduped = df.drop_duplicates(
    subset=['exp_name', 'task', 'ring_type', 'method_name', 'mae_with_std', 'rmse_with_std', 
            'mape_with_std', 'pearson_with_std', 'mode'],
    keep='first'
)

# 5. 保存去重数据
df_deduped.to_csv(os.path.join(output_dir, 'deduped_results.csv'), index=False, encoding='utf-8-sig')
print(f"\n去重后记录数：{len(df_deduped)}")
print("\n去重后方法分布：")
print(df_deduped['method_name'].value_counts())
print("\n去重后 test 模式前几行：")
print(df_deduped[df_deduped['mode'] == 'test'][['exp_name', 'task', 'ring_type', 'method_name', 'mae_with_std']].head(10))
print("\n去重后 5fold 模式前几行：")
print(df_deduped[df_deduped['mode'] == '5fold'][['exp_name', 'task', 'ring_type', 'method_name', 'mae_with_std']].head(10))


去重后记录数：110

去重后方法分布：
peak              36
fft               36
transformer       12
inception_time    12
resnet            12
ratio              2
Name: method_name, dtype: int64

去重后 test 模式前几行：
       exp_name            task ring_type method_name mae_with_std
4   peak-motion              hr     ring2        peak   41.40±3.59
5   peak-motion              hr     ring1        peak   18.75±1.84
6   peak-motion      samsung_hr     ring2        peak   35.96±4.01
7   peak-motion      samsung_hr     ring1        peak   16.47±6.32
8   peak-motion         oura_hr     ring2        peak   35.96±4.01
9   peak-motion         oura_hr     ring1        peak   16.47±6.32
10  peak-motion  samsung_hr_com     ring2        peak  39.87±10.55
11  peak-motion  samsung_hr_com     ring1        peak   43.72±9.72
12  peak-motion         resp_rr     ring2        peak    4.54±0.50
13  peak-motion         resp_rr     ring1        peak    4.60±0.44

去重后 5fold 模式前几行：
                           exp_name     task rin

In [3]:
# 6. 提取均值和标准差
def extract_mean_std(col):
    try:
        matches = col.str.extract(r'(\d+\.\d+|\d+)±(\d+\.\d+|\d+)')
        return matches.astype(float).rename(columns={0: 'mean', 1: 'std'})
    except:
        return pd.DataFrame({'mean': np.nan, 'std': np.nan})

df_deduped[['mae', 'mae_std']] = extract_mean_std(df_deduped['mae_with_std'])
df_deduped[['rmse', 'rmse_std']] = extract_mean_std(df_deduped['rmse_with_std'])
df_deduped[['mape', 'mape_std']] = extract_mean_std(df_deduped['mape_with_std'])
df_deduped[['pearson', 'pearson_std']] = extract_mean_std(df_deduped['pearson_with_std'])

# 7. 分组
grouped = df_deduped.groupby(['task', 'ring_type', 'method_name', 'mode'])[['mae', 'rmse', 'mape', 'pearson']].mean().reset_index()
grouped.to_csv(os.path.join(output_dir, 'results_grouped.csv'), index=False, encoding='utf-8-sig')
print(f"\n分组后记录数：{len(grouped)}")
print("\n分组后方法分布：")
print(grouped['method_name'].value_counts())

# 8. 去重分组数据
grouped_deduped = grouped.drop_duplicates(
    subset=['task', 'ring_type', 'method_name', 'mae', 'rmse', 'mape', 'pearson'],
    keep='first'
)
grouped_deduped.to_csv('results_grouped_deduped.csv', index=False, encoding='utf-8-sig')
print(f"\n分组去重后记录数：{len(grouped_deduped)}")
print("\n分组去重后方法分布：")
print(grouped_deduped['method_name'].value_counts())


分组后记录数：44

分组后方法分布：
fft               12
peak              12
inception_time     6
resnet             6
transformer        6
ratio              2
Name: method_name, dtype: int64

分组去重后记录数：44

分组去重后方法分布：
fft               12
peak              12
inception_time     6
resnet             6
transformer        6
ratio              2
Name: method_name, dtype: int64


/tmp/ipykernel_4124704/1720670309.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_deduped[['mae', 'mae_std']] = extract_mean_std(df_deduped['mae_with_std'])
/tmp/ipykernel_4124704/1720670309.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_deduped[['mae', 'mae_std']] = extract_mean_std(df_deduped['mae_with_std'])
/tmp/ipykernel_4124704/1720670309.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value ins

In [4]:
# 9. 最佳性能
best_results = grouped_deduped.groupby(['task', 'ring_type', 'mode']).apply(
    lambda x: x.loc[x['mae'].idxmin()]
).reset_index(drop=True)
best_results.to_csv(os.path.join(output_dir, 'best_results.csv'), index=False, encoding='utf-8-sig')
print("\n最佳性能（按任务、戒指类型和模式）：")
print(best_results[['task', 'ring_type', 'mode', 'method_name', 'mae']])


最佳性能（按任务、戒指类型和模式）：
              task ring_type   mode     method_name        mae
0               hr     ring1  5fold  inception_time   9.930000
1               hr     ring1   test     transformer  13.003333
2               hr     ring2  5fold  inception_time  10.190000
3               hr     ring2   test     transformer  13.660000
4          oura_hr     ring1   test            peak  17.040000
5          oura_hr     ring2   test            peak  19.093333
6      oura_hr_com     ring1   test             fft  17.590000
7      oura_hr_com     ring2   test             fft  19.350000
8          resp_rr     ring1  5fold     transformer   3.260000
9          resp_rr     ring1   test            peak   3.630000
10         resp_rr     ring2   test            peak   3.953333
11      samsung_hr     ring1   test            peak  15.236667
12      samsung_hr     ring2   test            peak  21.496667
13  samsung_hr_com     ring1   test             fft  19.336667
14  samsung_hr_com     ring2   test

In [5]:

# 11. MAE 柱状图
tasks = df_deduped['task'].unique()
for task in tasks:
    for mode in ['test', '5fold']:
        task_mode_data = df_deduped[(df_deduped['task'] == task) & (df_deduped['mode'] == mode)]
        if task_mode_data.empty:
            continue
        plt.figure(figsize=(15, 8))
        ax = sns.barplot(
            data=task_mode_data,
            x='method_name',
            y='mae',
            hue='ring_type',
            dodge=True,
            errorbar=None,
            palette='Set2'
        )
        for p in ax.patches:
            height = p.get_height()
            if not pd.isna(height) and height > 0:
                ax.text(
                    p.get_x() + p.get_width() / 2.,
                    height + 0.005 * max(task_mode_data['mae'].fillna(0)) + 0.1,
                    f'{height:.2f}',
                    ha='center', va='bottom', fontsize=10, color='black'
                )
        plt.title(f'{task.upper()} - MAE Comparison ({mode} Mode)')
        plt.xlabel('Method')
        plt.ylabel('MAE')
        plt.xticks(rotation=45)
        plt.legend(title='Ring Type')
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, f'mae_barplot_{task}_{mode}.png'), dpi=300)
        plt.close()
print(f"\n已生成 MAE 柱状图，保存到 {output_dir}/mae_barplot_<task>_<mode>.png")

# 12. MAE 热力图
for mode in ['test', '5fold']:
    mode_data = grouped_deduped[grouped_deduped['mode'] == mode]
    pivot_mae = mode_data.pivot_table(
        index='task',
        columns=['ring_type', 'method_name'],
        values='mae',
        aggfunc='mean'
    )
    plt.figure(figsize=(12, 6))
    sns.heatmap(pivot_mae, annot=True, fmt='.2f', cmap='YlOrRd')
    plt.title(f'MAE Heatmap ({mode} Mode)')
    plt.xlabel('Method (by Ring Type)')
    plt.ylabel('Task')
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, f'mae_heatmap_{mode}.png'), dpi=300)
    plt.close()
print(f"\n已生成 MAE 热力图，保存到 {output_dir}/mae_heatmap_<mode>.png")


已生成 MAE 柱状图，保存到 output/mae_barplot_<task>_<mode>.png

已生成 MAE 热力图，保存到 output/mae_heatmap_<mode>.png
